In [2]:
from src.data.data_loader import load_data

# from src.data.data_transforms import create_time_diff_cols
from sktime.transformations.series.impute import Imputer

df = load_data(use_cached=True)
# df = create_time_diff_cols(df)

df.columns

Index(['id', 'p_num', 'time', 'bg-0:00', 'insulin-0:00', 'carbs-0:00',
       'hr-0:00', 'steps-0:00', 'cals-0:00', 'activity-0:00', 'cob',
       'carb_availability', 'datetime', 'insulin_availability', 'iob'],
      dtype='object')

### Use patient 1

In [3]:
patient_1 = df[df["p_num"] == "p01"]
print(len(patient_1))
full_len = len(patient_1)
# Display full dataframe
patient_1

8459


,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,cob,carb_availability,datetime,insulin_availability,iob
0,p01_0,p01,06:10:00,15.1,0.0417,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2025-01-01 06:10:00,0.000000,0.041700
1,p01_1,p01,06:25:00,14.4,0.0417,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2025-01-01 06:25:00,0.000646,0.082961
2,p01_2,p01,06:40:00,13.9,0.0417,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2025-01-01 06:40:00,0.002091,0.122052
3,p01_3,p01,06:55:00,13.8,0.0417,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2025-01-01 06:55:00,0.003964,0.157680
4,p01_4,p01,07:10:00,13.4,0.0417,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2025-01-01 07:10:00,0.005981,0.189253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8454,p01_8454,p01,22:50:00,6.2,0.0333,NaN,76.8,70.0,5.00,NaN,0.0,0.0,2025-04-01 22:50:00,0.012917,0.247948
8455,p01_8455,p01,23:05:00,5.8,0.0167,NaN,72.2,13.0,5.00,NaN,0.0,0.0,2025-04-01 23:05:00,0.012917,0.231348
8456,p01_8456,p01,23:20:00,5.4,0.0167,NaN,73.4,18.0,5.10,NaN,0.0,0.0,2025-04-01 23:20:00,0.012659,0.214922
8457,p01_8457,p01,23:35:00,4.8,0.0167,NaN,64.7,0.0,7.89,NaN,0.0,0.0,2025-04-01 23:35:00,0.012084,0.199361


### Reduce the size of the df

In [4]:
# We'll just tune the hyperparams on the first 100 timepoints for patient 1 since we're just testing
# full is 8459
row_to_load = 1000
cols = ["bg-0:00", "iob", "cob"]

patient_1_trimmed = df.iloc[:row_to_load][cols]
patient_1_trimmed

,bg-0:00,iob,cob
0,15.1,0.041700,0.0
1,14.4,0.082961,0.0
2,13.9,0.122052,0.0
3,13.8,0.157680,0.0
4,13.4,0.189253,0.0
...,...,...,...
995,7.2,3.239977,0.0
996,6.7,2.870734,0.0
997,6.2,2.493940,0.0
998,5.1,2.134549,0.0


### Handle missing values 

In [5]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.split import ExpandingWindowSplitter, temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanSquaredError


def load_diabetes():
    transform = Imputer(method="linear")
    y = patient_1_trimmed.copy()
    y["bg-0:00"] = transform.fit_transform(y[["bg-0:00"]])
    return (y[["bg-0:00"]], y[["iob", "cob"]])


y_diabetes, X_diabetes = load_diabetes()

c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\.venv\Lib\site-packages\gluonts\json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


### Benchmark
With out current setup for testing, our dataset is `patient_1_trimmed`, which only has 200 rows with 2 X features: iob and cob.

In [6]:
from src.tuning.benchmark import generate_estimators_from_param_grid
from datetime import datetime
import os

benchmark = ForecastingBenchmark()

# Some models don't like from 0 to 72, so maybe need to start from 1?
cv_splitter = ExpandingWindowSplitter(
    initial_window=360,
    step_length=72,
    fh=[1, 2, 3, 4],
)

# cv_splitter = temporal_train_test_split(y_diabetes, fh=24)

# PinballLoss is giving us some issues
scorers = [MeanSquaredError(square_root=True)]

benchmark.add_task(
    dataset_loader=load_diabetes,
    cv_splitter=cv_splitter,
    scorers=scorers,
    # error_score="raise",
)

## Change your yaml path here

In [7]:
import pandas as pd

## Change your yaml path here
yaml_path = "../../src/tuning/configs/4_dl_ts_15min.yaml"

estimators = generate_estimators_from_param_grid(yaml_path)
for estimator, estimator_id in estimators:
    benchmark.add_estimator(estimator=estimator, estimator_id=estimator_id)

# Needs to add the datetime to make the file name unique
# so when you run benchmark.run, it doesn't see the file as cached result and not rerun
current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
yaml_name = yaml_path.split("/")[-1].replace(".yaml", "")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = f"../results/param_tests/{current_time}_results_{yaml_name}.csv"
benchmark.run(results_file)

# If there is a file created, then it is probably good enough
if not os.path.exists(results_file):
    raise FileNotFoundError(f"Results file {results_file} was not created")

Training 1 NeuralForecastRNN models with different parameters


c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-21 10:11:29,846	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-21 10:11:30,250	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Seed set to 1
c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\.venv\Lib\site-packages\neuralforecast\common\_base_model.py:349: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    

Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s, v_num=31, train_loss_step=0.971, train_loss_epoch=0.971]

`Trainer.fit` stopped: `max_steps=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s, v_num=31, train_loss_step=0.971, train_loss_epoch=0.971]

...

`Trainer.fit` stopped: `max_steps=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:03<00:00,  0.30it/s, v_num=47, train_loss_step=0.971, train_loss_epoch=0.971]


c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\.venv\Lib\site-packages\neuralforecast\core.py:904: UserWarning: Dropped 936 unused rows from `futr_df`.
  warnings.warn(f"Dropped {dropped_rows:,} unused rows from `futr_df`.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


OSError: Cannot save file into a non-existent directory: 'results\param_tests'

In [53]:
# importing necessary libraries
from sktime.forecasting.neuralforecast import NeuralForecastRNN

# loading the Longley dataset and splitting it into train and test subsets
y, X = load_diabetes()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=4)

In [62]:
# creating model instance configuring the hyperparameters
# NOTE: frequency 1 indicates frequency the data is collected. Our data is not timestamped by time (it is just indexed like a list. Se set frequency to 1)
model = NeuralForecastRNN(freq=1, futr_exog_list=["iob", "cob"], max_steps=5)
# fitting the model
model.fit(y_train, X=X_train, fh=[1, 2, 3, 4])
# getting point predictions
model.predict(X=X_test)

Seed set to 1


c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\.venv\Lib\site-packages\neuralforecast\common\_base_model.py:349: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | RNN           | 121 K  | train
4 | context_adapter | Linear        | 8.4 K  | train
5 | mlp_decoder     | MLP           | 2.8 K  | train
----------------------------------------------------------
132 K     Trainable params
0         Non-trainable params
132 K     Total params
0.529     Total estimated model params size (MB)
11        Modules in train mode
0 

Epoch 4: 100%|██████████| 1/1 [00:02<00:00,  0.38it/s, v_num=11, train_loss_step=0.985, train_loss_epoch=0.985]

`Trainer.fit` stopped: `max_steps=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:02<00:00,  0.37it/s, v_num=11, train_loss_step=0.985, train_loss_epoch=0.985]

GPU available: False, used: False


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


,bg-0:00
996,9.167429
997,8.690634
998,8.517226
999,7.152248
